In [1]:
#from sklearn.preprocessing import LabelEncoder
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder

# Sem WHOQOL:
base1 = pd.read_csv('Datasets/dataframe_socioeconomico_Niveis.csv')
data1 = base1.drop('Nivel_MHI', axis=1)
target1 = base1['Nivel_MHI']
le = LabelEncoder()
target1 = le.fit_transform(target1)

# Realizar Random Under-Sampling
sampler = RandomUnderSampler()
data1_re, target1_re = sampler.fit_resample(data1, target1)


# com todas as respostas do WHOQOL
base2 = pd.read_csv('Datasets/df_social_whoqol_tratado_1.csv')
data2 = base2.drop('Nivel_MHI', axis=1)
target2 = base2['Nivel_MHI']
# target2 = le.fit_transform(target2)
sampler = RandomUnderSampler()
data2_re, target2_re = sampler.fit_resample(data2, target2)


# só com o indice dos dominiosdo WHOQOL, concatenar base1 com final da base2
data3 = pd.concat([data1, base2.loc[:,['CR','CS','CT','CU']]], axis=1)
target3 = base2['Nivel_MHI']
target3 = le.fit_transform(target3)
sampler = RandomUnderSampler()
data3_re, target3_re = sampler.fit_resample(data3, target3)


# Só com a qualidade de vida do WHOQOL
data4 = pd.concat([data1, base2['BR']], axis=1)
target4 = base2['Nivel_MHI']
target4 = le.fit_transform(target4)
sampler = RandomUnderSampler()
data4_re, target4_re = sampler.fit_resample(data4, target4)

# Qualidade de Vida Geral ( Media das duas primeiras questões do WHOQOL)
# Pegar a média das duas primeiras colunas da base2, pelo indice
data5 = pd.concat([data1, base2.loc[:,['BR','BS']].mean(axis=1)], axis=1)
target5 = base2['Nivel_MHI']
target5 = le.fit_transform(target5)
sampler = RandomUnderSampler()
data5.columns = data5.columns.astype(str)
data5_re, target5_re = sampler.fit_resample(data5, target5)

# Media de todos os dominios do WHOQOL
data6 = pd.concat([data1, base2.loc[:,['CR','CS','CT','CU']].mean(axis=1)], axis=1)
target6 = base2['Nivel_MHI']
target6 = le.fit_transform(target6)
sampler = RandomUnderSampler()
data6.columns = data6.columns.astype(str)
data6_re, target6_re = sampler.fit_resample(data6, target6)

In [30]:
import pandas as pd
import numpy as np

# Função para calcular a entropia de um conjunto de dados
def entropy(data):
    class_labels, counts = np.unique(data, return_counts=True)
    total_instances = len(data)
    probabilities = counts / total_instances
    entropy_val = -np.sum(probabilities * np.log2(probabilities))

    return entropy_val

# Função para calcular o ganho de informação de um atributo
def information_gain(data, attribute, target):
    attribute_values = data[attribute].unique()
    total_instances = len(data)
    info_gain = entropy(data[target])

    for value in attribute_values:
        subset = data[data[attribute] == value]
        prob = len(subset) / total_instances
        info_gain -= prob * entropy(subset[target])

    return info_gain

# Função para calcular os ganhos de informação de todos os atributos em relação ao atributo alvo
def calculate_information_gains(data, target):
    all_attributes = data.columns.drop(target)
    gains = {attribute: information_gain(data, attribute, target) for attribute in all_attributes}
    result_df = pd.DataFrame(gains.items(), columns=['Atributo', 'Ganho de Informação'])
    result_df = result_df.sort_values(by='Ganho de Informação', ascending=False)
    return result_df

# Função para ler o arquivo "titulos_das_colunas.txt" e criar um dicionário com o mapeamento dos índices para os nomes das colunas
def read_column_titles(file_path):
    column_titles = {}
    questionarios = {}
    with open(file_path, 'r', encoding='ISO-8859-1') as file:
        lines = file.readlines()
        i = 0
        while i < len(lines):
            if lines[i].startswith("Título da Coluna:"):
                column_title = lines[i].split(":")[1].strip()
                coordinate = lines[i+1].split(":")[1].strip()
                questionario = lines[i+2].split(":")[1].strip()
                column_titles[coordinate] = column_title
                questionarios[coordinate] = questionario
                
                i += 2
            else:
                i += 1
    return column_titles, questionarios


# Ler o dataframe e pré-processar o atributo alvo
base1 = pd.read_csv('Datasets/dataframe_socioeconomico_Niveis.csv')
data1 = base1.drop('Nivel_MHI', axis=1)
target1 = base1['Nivel_MHI']
le = LabelEncoder()
target1 = le.fit_transform(target1)

# Converter o atributo alvo de volta para rótulos de classes
target_labels = le.inverse_transform(target1)
data1['Nivel_MHI'] = target_labels


# Calcular os ganhos de informação
result_df = calculate_information_gains(data1, 'Nivel_MHI')

# Imprimir o resultado
print(result_df)

   Atributo  Ganho de Informação
26       BP             0.055492
1         C             0.039201
23       AZ             0.036363
7         O             0.022827
13       AO             0.021020
5         M             0.020295
14       AP             0.017528
44    J_5.0             0.016862
19       AV             0.015995
25       BO             0.015834
8         U             0.014742
24       BD             0.014318
3         H             0.012816
4         L             0.012538
16       AS             0.011770
40    J_1.0             0.011435
12       AN             0.009764
33    E_2.0             0.008947
0         B             0.008395
32    E_1.0             0.006683
15       AR             0.006624
21       AX             0.005518
45   AA_1.0             0.004869
39    I_4.0             0.003939
6         N             0.003749
20       AW             0.003630
42    J_3.0             0.003394
17       AT             0.002942
22       AY             0.002646
36    I_1.

In [31]:



# Ler o arquivo "titulos_das_colunas.txt" e criar o dicionário de mapeamento dos índices para os nomes das colunas
column_titles,questionarios = read_column_titles('titulos_das_colunas.txt')

# com todas as respostas do WHOQOL
base2 = pd.read_csv('Datasets/df_social_whoqol_tratado_1.csv')
data2 = base2.drop('Nivel_MHI', axis=1)
target2 = base2['Nivel_MHI']
target2 = le.fit_transform(target2)

# Converter o atributo alvo de volta para rótulos de classes
target_labels = le.inverse_transform(target2)
data2['Nivel_MHI'] = target_labels

In [32]:
result_df

,Atributo,Ganho de Informação
26,BP,0.055492
1,C,0.039201
23,AZ,0.036363
7,O,0.022827
13,AO,0.021020
5,M,0.020295
14,AP,0.017528
44,J_5.0,0.016862
19,AV,0.015995
25,BO,0.015834


In [33]:
# Calcular os ganhos de informação
result_df = calculate_information_gains(data2, 'Nivel_MHI')


In [34]:
# Traduzir os índices para os nomes originais das colunas
result_df['Questionario'] = result_df['Questionario'].map(questionarios)
result_df

KeyError: 'Questionario'

In [ ]:
# Calcular ganho de informação da tabela e a soma dos ganhos de informação.
result_df['Ganho de Informação'] = result_df['Ganho de Informação'].astype(float)


In [ ]:
result_df

,Atributo,Ganho de Informação,Coordenadas,Questionario
81,DOMINIO 2(PSICOLOGICO),0.496086,CS,WHOQOL
80,DOMINIO 1(FISICO),0.338931,CR,WHOQOL
79,Com que frequência você tem sentimentos negati...,0.317214,CQ,WHOQOL
72,Quão satisfeito(a) você está consigo mesmo?,0.309827,CJ,WHOQOL
61,Quão seguro(a) você se sente em sua vida diária?,0.267769,BY,WHOQOL
...,...,...,...,...
2,Local de origem(Sobral),0.000224,F,Sociodemografico
51,Transporte para à Universidade(Transporte cole...,0.000197,AB_2.0,Sociodemografico
18,Atividade de Lazer(Assitir filmes/ séries),0.000149,AU,Sociodemografico
34,Situação Civil(União estável),0.000102,E_3.0,Sociodemografico


In [ ]:
# Calculate the entropy of the whole database
entropy_whole_db = entropy(data1['Nivel_MHI'])
print("Entropy of the whole database:", entropy_whole_db)


Entropy of the whole database: 1.2682201103164115


In [ ]:
sum_of_gains = result_df['Ganho de Informação'].sum()
print("Sum of information gains:", sum_of_gains)


Sum of information gains: 5.455830720408816


In [5]:
import pandas as pd
import math

def entropy(column):
    """Calculate the entropy of a given column."""
    unique_values = column.value_counts().index
    total_rows = len(column)
    entropy_value = 0

    for value in unique_values:
        p = len(column[column == value]) / total_rows
        entropy_value -= p * math.log2(p) if p > 0 else 0

    return entropy_value

def information_gain(data, target_column, attribute_column):
    """Calculate the information gain for a specific attribute column."""
    total_entropy = entropy(data[target_column])
    unique_values = data[attribute_column].value_counts().index

    weighted_entropy = 0
    for value in unique_values:
        subset = data[data[attribute_column] == value]
        subset_entropy = entropy(subset[target_column])
        weight = len(subset) / len(data)
        weighted_entropy += weight * subset_entropy

    info_gain = total_entropy - weighted_entropy
    return info_gain / entropy(data[attribute_column])

def calculate_entropy_and_gain(data, target_column):
    """Calculate entropy and normalized information gain for all attribute columns in the dataframe."""
    attribute_columns = data.columns.difference([target_column])
    results = []

    for attribute_column in attribute_columns:
        info_gain = information_gain(data, target_column, attribute_column)
        entropy_value = entropy(data[attribute_column])

        results.append({
            'Attribute': attribute_column,
            'Entropy': entropy_value,
            'Information_Gain': info_gain
        })

    result_df = pd.DataFrame(results)
    return result_df


# Example usage:
# Assuming df is your dataframe and 'Nivel_MHI' is your target column
result_df = calculate_entropy_and_gain(base2, 'Nivel_MHI')

#Sort the dataframe by information gain
result_df = result_df.sort_values(by='Information_Gain', ascending=False)

# Display the results dataframe
print(result_df)


# Ler o arquivo "titulos_das_colunas.txt" e criar o dicionário de mapeamento dos índices para os nomes das colunas
column_titles,questionarios = read_column_titles('titulos_das_colunas.txt')



   Attribute   Entropy  Information_Gain
54        CQ  1.995563          0.158960
47        CJ  2.091360          0.148146
35        BY  1.898338          0.141054
56        CS  4.148223          0.119590
33        BW  2.042703          0.112643
..       ...       ...               ...
9         AC  0.906809          0.000304
68         F  0.923185          0.000242
6     AB_2.0  0.993612          0.000198
18        AU  0.978549          0.000152
61     D_3.0  0.805780          0.000082

[84 rows x 3 columns]


NameError: name 'read_column_titles' is not defined

In [37]:

#Copiando Coordenadas
result_df['Coordenadas'] = result_df['Attribute']

# Copiar "Atributo" para poder mapear para questões
result_df['Questionario'] = result_df['Coordenadas']

# Traduzir os índices para os nomes originais das colunas
result_df['Attribute'] = result_df['Attribute'].map(column_titles)

result_df['Questionario'] = result_df['Questionario'].map(questionarios)

result_df

,Attribute,Entropy,Information_Gain,Coordenadas,Questionario
54,Com que frequência você tem sentimentos negati...,1.995563,0.158960,CQ,WHOQOL
47,Quão satisfeito(a) você está consigo mesmo?,2.091360,0.148146,CJ,WHOQOL
35,Quão seguro(a) você se sente em sua vida diária?,1.898338,0.141054,BY,WHOQOL
56,DOMINIO 2(PSICOLOGICO),4.148223,0.119590,CS,WHOQOL
33,Em que medida você acha que a sua vida tem sen...,2.042703,0.112643,BW,WHOQOL
...,...,...,...,...,...
9,Realiza atividades profissionais?,0.906809,0.000304,AC,Sociodemografico
68,Local de origem(Sobral),0.923185,0.000242,F,Sociodemografico
6,Transporte para à Universidade(Transporte cole...,0.993612,0.000198,AB_2.0,Sociodemografico
18,Atividade de Lazer(Assitir filmes/ séries),0.978549,0.000152,AU,Sociodemografico


In [41]:
# Calcular ganho de informação da tabela e a soma dos ganhos de informação.
result_df['Information_Gain'] = result_df['Information_Gain'].astype(float)

sum_of_gains = result_df['Information_Gain'].sum()
print("Sum of information gains:", sum_of_gains)

# Calculate the entropy of the whole database
entropy_whole_db = entropy(base2['Nivel_MHI'])
print("Entropy of the whole database:", entropy_whole_db)

Sum of information gains: 2.9290189460491463
Entropy of the whole database: 1.2682201103164115


In [6]:
import pandas as pd
import numpy as np

def calculate_entropy_ig(df, target_col="Nivel_MHI"):
  """
  Calculates the entropy and information gain of a dataframe for a multiclass classification problem.

  Args:
    df: A pandas dataframe containing the data.
    target_col: The name of the column containing the target variable.

  Returns:
    A dictionary containing the entropy and information gain for each attribute in the dataframe.
  """

  # Calculate entropy of the target variable
  entropy = calculate_entropy(df[target_col])

  # Calculate information gain for each attribute
  ig_dict = {}
  for col in df.columns:
    if col != target_col:
      ig_dict[col] = calculate_information_gain(df, col, target_col, entropy)

  return ig_dict

def calculate_entropy(data):
  """
  Calculates the entropy of a given data series.

  Args:
    data: A pandas series or numpy array containing the data.

  Returns:
    The entropy of the data.
  """

  p = data.value_counts(normalize=True)  # Calculate probabilities of each class
  entropy = -np.sum(p * np.log2(p))  # Apply entropy formula
  return entropy

def calculate_information_gain(df, attribute, target_col, entropy):
  """
  Calculates the information gain of a given attribute for a multiclass classification problem.

  Args:
    df: A pandas dataframe containing the data.
    attribute: The name of the attribute to calculate the information gain for.
    target_col: The name of the column containing the target variable.
    entropy: The entropy of the target variable.

  Returns:
    The information gain of the attribute.
  """

  weighted_entropy = 0
  for value in df[attribute].unique():
    subset = df[df[attribute] == value]  # Split data based on attribute value
    p = subset[target_col].value_counts(normalize=True)
    weighted_entropy += (len(subset) / len(df)) * calculate_entropy(p)  # Calculate weighted entropy
  return entropy - weighted_entropy  # Information gain is the reduction in entropy


# Example usage:

# Assuming df is your dataframe and 'Nivel_MHI' is your target column

result_df = calculate_entropy_ig(base2, 'Nivel_MHI')

# Sort the dataframe by information gain

result_df = pd.DataFrame(result_df.items(), columns=['Atributo', 'Ganho de Informação'])

result_df = result_df.sort_values(by='Ganho de Informação', ascending=False)

# Display the results dataframe

print(result_df)

   Atributo  Ganho de Informação
35    E_4.0             0.350977
49   AB_0.0             0.350977
48   AA_4.0             0.349924
6         N             0.349924
27    D_1.0             0.349924
..      ...                  ...
71       CI            -0.316742
52   AB_3.0            -0.316742
32    E_1.0            -0.316742
39    I_4.0            -0.316742
24       BD            -0.316742

[84 rows x 2 columns]
